In [42]:
import itertools
import random
import simpy

RANDOM_SEED = 42
ROOM_CLEANING_SIZE = 100       # máximo nive de limpieza de una habitación 
THRESHOLD = 50                 # mínimo de limpieza/confort (% del total)
TOURIST_ENERGY_SIZE = 50       # nivel máximo de descanso
TOURIST_ENERGY_LEVEL = [5, 25] # nivel de energía de los turistas (menor_energía => más_sueño)
SLEEP_SPEED = 2                # tasa de recuperar energía (u / second)
HOUSEMAID_TIME = 300           # tiempo que tarda la mucama en limpiar la habitación (segundos)
T_INTER = [30, 300]            # intervalo entre la llegada de los turistas
SIM_TIME = 1000                # tiempo total de la simulación



SALARIES_AMOUNT = {} # salario cobrado por cada trabajador
SALARIES_AMOUNT['housemaid'] = 0
AMOUNT = {} # pago acumulado por os turistas en cada servicio
AMOUNT['room'] = 0

#### Definiendo precios

In [43]:
PRICES = {}
PRICES['room'] = 50

SALARIES = {}
SALARIES['housemaid'] = 5

### Definiendo los resultados de la red social

In [44]:
SATISFACTION = {}

# Agents
## Tourist

In [29]:
def tourist(name, env, room, bed):
    """A car arrives at the gas station for refueling.

    It requests one of the gas station's fuel pumps and tries to get the
    desired amount of fuel from it. If the station's fuel tank is
    depleted, the car has to wait for the tank truck to arrive.

    """
    tourist_energy_level = random.randint(*TOURIST_ENERGY_LEVEL)

    print(f'{env.now:6.1f} s: {name} arrived at room')
    with room.request() as req:
        # Request one of the rooms
        yield req

        AMOUNT['room'] += PRICES['room']

        # Get the required amount of energy
        energy_required = TOURIST_ENERGY_SIZE - tourist_energy_level
        yield bed.get(energy_required)

        # The "actual" recovering of energy process takes some time
        yield env.timeout(energy_required / SLEEP_SPEED)

        print(f'{env.now:6.1f} s: {name} recovered with {energy_required:.1f}units of sleep')

        if name not in SATISFACTION:
            SATISFACTION[name] = {}
        
        SATISFACTION[name]['energy'] = energy_required + tourist_energy_level

        print(f'Actual level of satisfaction: {SATISFACTION}\n Actual amount: {AMOUNT}')
        print(f'Actual level of clean of room: {bed.level}')
        
    


## Workers

In [15]:
def housemaid(env, bed):
    """Arrives at the room and finish of clean it after a certain delay."""
    #add si no tiene el salario completo no rellena al máximo el nivel de limpieza
    yield env.timeout(HOUSEMAID_TIME)
    amount = bed.capacity - bed.level
    bed.put(amount)

    print(
        f'{env.now:6.1f} s: Housemaid finished and the room is clean'
    )

## Manager

In [39]:
from deepdiff import DeepDiff

def manager(env, bed):
    """Periodically check the clean level of the room and call the housemaid
       if the level falls below a threshold."""
    while True:
        # DO SOMETHING FOR VERIFY THE EVOLUTION OF THE AMOUNT%%%%%%%%%%%%%%%%%%%%$$$$$$$$$$$$$$$$###################


        # actual_amount = AMOUNT
        # yield env.timeout(30)
        # diff = DeepDiff(actual_amount, AMOUNT)
        # print(f'Actualization of evolution of amount: {diff}')

        if bed.level / bed.capacity * 100 < THRESHOLD:
            # We need to call the housemaid now!
            print(f'{env.now:6.1f} s: Housemaid is cleaning the room')
            # Wait for the housemaid to clean the room
            yield env.process(housemaid(env, bed))

            housemaid_salary = SALARIES['housemaid']

            if AMOUNT['room'] >= housemaid_salary:
                AMOUNT['room'] -= housemaid_salary
                SALARIES_AMOUNT['housemaid'] += housemaid_salary
                print(f'{env.now:6.1f} s: The housemaid charaged ${housemaid_salary}. The amount salary of housemaid is {SALARIES_AMOUNT['housemaid']}')

            # else: (algo con el rendimiento de la housemaid al limpiar la habitación)
        
        yield env.timeout(5)  # Check every 10 seconds

        # añadir análisis con las ganancias

In [17]:
def tourist_generator(env, room, bed):
    """Generate new tourists that arrive at the hotel."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(tourist(f'Tourist {i}', env, room, bed))

# Simulation

In [46]:
# Setup and start the simulation
print('Hotel is open')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()  # hotel

# modificar room para que sea un array de reosurces con capacidad 1, luego habrá que modificar el resto de las
# func que utilizan room para que lo traten como un array
room = simpy.Resource(env, 2)

bed = simpy.Container(env, ROOM_CLEANING_SIZE, init=ROOM_CLEANING_SIZE)
env.process(manager(env, bed))
env.process(tourist_generator(env, room, bed))

# Execute!
env.run(until=SIM_TIME)

Hotel is open
  87.0 s: Tourist 0 arrived at room
 105.5 s: Tourist 0 recovered with 37.0units of sleep
Actual level of satisfaction: {'Tourist 0': {'energy': 50}, 'Tourist 1': {'energy': 50}, 'Tourist 3': {'energy': 50}, 'Tourist 2': {'energy': 50}, 'Tourist 5': {'energy': 50}, 'Tourist 4': {'energy': 50}}
 Actual amount: {'room': 440}
Actual level of clean of room: 63
 129.0 s: Tourist 1 arrived at room
 130.0 s: Housemaid is cleaning the room
 148.0 s: Tourist 1 recovered with 38.0units of sleep
Actual level of satisfaction: {'Tourist 0': {'energy': 50}, 'Tourist 1': {'energy': 50}, 'Tourist 3': {'energy': 50}, 'Tourist 2': {'energy': 50}, 'Tourist 5': {'energy': 50}, 'Tourist 4': {'energy': 50}}
 Actual amount: {'room': 490}
Actual level of clean of room: 25
 284.0 s: Tourist 2 arrived at room
 385.0 s: Tourist 3 arrived at room
 430.0 s: Housemaid finished and the room is clean
 430.0 s: The housemaid charaged $5. The amount salary of housemaid is 15
 435.0 s: Housemaid is cleanin

## Gestión de pagos/cobros

In [19]:


def cliente(env, nombre_cliente):
    print(f'{nombre_cliente} llega al mercado')
    for i in range(3):  # Suponemos que el cliente va a visitar a 3 vendedores
        yield env.timeout(1)  # Espera un tiempo antes de ir al siguiente vendedor
        print(f'{nombre_cliente} está hablando con el vendedor {i+1}')
        monto = 100 * (i + 1)  # El monto a pagar aumenta con cada vendedor
        print(f'{nombre_cliente} le pide al vendedor {i+1} {monto} euros')
        
        # Actualizamos el estado del vendedor con el monto recibido
        vendedores_estado[f'Vendedor {i+1}'] += monto

def vendedor(env, nombre_vendedor):
    print(f'{nombre_vendedor} abre su puesto')
    yield env.timeout(0.5)  # Tiempo que el vendedor espera por un cliente
    print(f'{nombre_vendedor} cierra su puesto')

    # Inicializamos el estado del vendedor si aún no se ha registrado
    if nombre_vendedor not in vendedores_estado:
        vendedores_estado[nombre_vendedor] = 0